# Session 04: Single-image processing

<br>Owner(s): **Andrew Bradshaw** ([@andrewkbradshaw](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@andrewkbradshaw))
<br>Last Verified to Run: **2020-06-04**
<br>Verified Stack Release: **w_2020_22**

This notebook is a translation of Andrew's slides from Session 04. It performs single-image processing using real spot data from the BOT. Andrew's original slides can be found [here](https://docs.google.com/presentation/d/13kYEhKHXIOBYqgHr5JxjLgLflyExptDVyEf8xtOWp-s/edit#slide=id.p).

In [ ]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s lsst_distrib

In [ ]:
%matplotlib inline

## Creating a local repository

In the Science Pipelines, you don’t directly manage data files. Instead, you access data through the Butler. This will hopefully give you flexibility to work with data from different observatories and non-local disks without significantly changing your workflow.

The Butler manages data in repositories. Butler repositories can be remote (the data is on a server, across a network) or local (the data in on a local filesystem). Here you’ll create and use a local Butler repository, which is a simple directory.

Below is how to create a local Butler repository in the DATA folder your home directory

In [ ]:
# This is where all the data exists at NCSA
DATA_DIR = '/project/shared/BOT/'

# Where to store your stuff (change YOURNAME to your username)
REPO_DIR = '/home/kadrlica/DATA/STACK_CLUB_BOT_TEST/' 
CALIB_DIR = REPO_DIR + "CALIB"
RERUN_DIR = REPO_DIR + "calib_construction"

# Create the REPO_DIR and add the mapper.

#! rm -rf {REPO_DIR} # Uncomment to remove exisitng directory
! mkdir -p {REPO_DIR}
! echo "lsst.obs.lsst.lsstCamMapper.LsstCamMapper" > {REPO_DIR+"_mapper"}
! mkdir -p {CALIB_DIR}

## Making a master bias frame

In [ ]:
# Now create a master bias for a detector 
RAFT, DETECTOR_NAME, DETECTOR= 'R22','S11',94

# use only 5 bias frames with visit numbers
start=3019103101866
end  =3019103101870

# compose the arguments for command line
args = DATA_DIR + " --calib " + CALIB_DIR + \
   	" --rerun " + RERUN_DIR + \
   	" --id visit=%d..%d detector=%d"%(start,end,DETECTOR) + \
   	" --batch-type=None" + " -c isr.doCrosstalk=False" + \
" --clobber-config"

print(args)

! constructBias.py {args}

## Look at the new master bias

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import glob

biasname=glob.glob(RERUN_DIR + "/bias/*/*.fits")[0]
print(biasname)
bias=fits.getdata(biasname)

plt.figure(figsize=(10,10))
plt.imshow(bias,vmin=-10,vmax=10)
plt.colorbar()
plt.title('master bias frame \n  raft,sensor='+RAFT+','+DETECTOR_NAME)

## Ingest the master bias frame

You can now populate the repository with our first new data, the master bias frame we made. The Pipelines’ ingestImages.py command (called a command-line task) links raw images into a Butler repository, allowing the mapper to organize the data. ingestCalibs.py adds these as files specifically as calibration images for other data from that sensor and time (validity range)

Notice that the first argument to most command-line tasks is the Butler repository. In this case it’s the `DATA` directory.

You can learn about the arguments for command-line tasks with the `-h` flag. For example:
```
ingestImages.py -h
```

In [ ]:
# Now ingest the master bias image
args = REPO_DIR + " " + RERUN_DIR + "/bias/*/*.fits" + " --validity 9999" + " --calib " + CALIB_DIR + " --mode=link"
! ingestCalibs.py {args}

## Now make a master dark

In [ ]:
# Now create a master dark
# first make a list of visits
# Five 30 second darks
visits = []
starting_visit = 3019103101871
ending_visit   = 3019103101875
visit = starting_visit
while visit < ending_visit + 1:
    visits.append(visit)
    visit += 1
print(len(visits))

In [ ]:
# create constructDark args string

args = DATA_DIR + " --calib " + CALIB_DIR + " --rerun " + RERUN_DIR + " --id visit="
for visit in visits:
    if visit != starting_visit:
        args += "^"
    args += str(visit)
args += " detector=%d"%DETECTOR
args += " --batch-type=None" + " -c isr.doCrosstalk=False " +  "--clobber-config"
print(args)
! constructDark.py {args}

## Ingest the master dark frame

In [ ]:
# Now ingest the master dark image
args = REPO_DIR + " " + \
       RERUN_DIR + "/dark/*/*.fits" + \
       " --validity 9999" + \
       " --calib " + CALIB_DIR + \
       " --mode=link"

! ingestCalibs.py {args}

## Similarly, make a master flat

In [ ]:
# Now create a master flat
visits = []
starting_visit = 3019110200307
ending_visit = 3019110200308

visit = starting_visit
while visit < ending_visit + 1:
    visits.append(visit)
    visit += 1
print(len(visits))

In [ ]:
args = DATA_DIR + " --calib " + CALIB_DIR + " --rerun " + RERUN_DIR + " --id visit="
for visit in visits:
    if visit != starting_visit:
        args += "^"
    args += str(visit)

args += " detector=%d"%DETECTOR
args += " --batch-type=None" + " -c isr.doCrosstalk=False" + " --clobber-config"

print(args)

! constructFlat.py {args}

## Ingest the master flat frame

In [ ]:
# Now ingest the master flat images
args = REPO_DIR + " " + RERUN_DIR + \
       "/flat/*/*/*.fits" + \
       " --validity 9999" + \
       " --calib " + CALIB_DIR + \
       " --mode=link"

! ingestCalibs.py {args}

## Now get a spot image to run ISR on

In [ ]:
from lsst.daf.persistence import Butler

spot_visit = 3019103101985  
#This is a medium exposure
butler = Butler(DATA_DIR, calibRoot=CALIB_DIR)
spotDataRef = butler.dataRef('raw', raftName=RAFT,
                         	detectorName=DETECTOR_NAME,
                         	visit=spot_visit)

## Configure an IsrTask and run it inside of python

In [ ]:
# configure an IsrTask
from lsst.ip.isr.isrTask import IsrTask
isrConfig = IsrTask.ConfigClass()
isrConfig.doLinearize = False
isrConfig.doBias = True     # use the calibs we made!
isrConfig.doFlat = True
isrConfig.doDark = True
isrConfig.doFringe = False   # we did not do these, skip
isrConfig.doDefect = False
isrConfig.doWrite = False
isrConfig.fallbackFilterName = 'SDSSi~empty'  #  Both of these are needed
isrConfig.useFallbackDate = True          	#  Both of these are needed
isrTask = IsrTask(config=isrConfig)
# run the task and take the exposure
postIsrSpot = isrTask.run(spotDataRef).exposure

## View the spot images after ISR

In [ ]:
# Now just look at a portion of the images
plt.figure(figsize=(16,16))    
plt.subplot(1,2,1)
plt.title("Post ISR Spot")
plt.imshow(postIsrSpot.image.array[2000:3000,2000:3000],vmin=0,vmax=1000)
plt.subplot(1,2,2)
plt.title("Post ISR Spot")
plt.imshow(postIsrSpot.image.array[2510:2550,2535:2575],vmin=0,vmax=1000)

## Find the spots in the corrected image

In [ ]:
# Now let's try finding the spots
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
charConfig = CharacterizeImageConfig()

charConfig.installSimplePsf.fwhm = 1.0   # save a step, start with a guess
charConfig.doMeasurePsf = False          # but you could measure it
charConfig.doApCorr = False              # a lot of nice steps we can skip
charConfig.doDeblend = False
charConfig.repair.doCosmicRay = True
charConfig.repair.doInterpolate = False   
charConfig.detection.background.binSize = 32    # what scale to est. bg
charConfig.detection.minPixels = 20             # min number of pixels
charTask = CharacterizeImageTask(config=charConfig)
charResult = charTask.run(postIsrSpot)

print("Detected ",len(charResult.sourceCat)," objects ")

## Display the spots found, using their centroid

In [ ]:
plt.figure(figsize=(16,16))
plt.title('X/Y locations of detections - E2V')
plt.plot(charResult.sourceCat['base_SdssCentroid_x'],
         charResult.sourceCat['base_SdssCentroid_y'],'r.')

## Playground...

Below is a place to play around with your spot image and catalog. For example, 

* Explore the catalog produced by the single fits image using data visualization tools
* Turn off bias and dark correction, or cosmic ray interpolation and re-do detection
* Try lowering the number of pixels for detection
* Look at the regions around cosmic rays.
* ...!

Add your own playground example...

## Playground: Investigating Cosmic Rays

In [ ]:
import numpy as np

# Figure out which pixels are identified as cosmic rays
print(postIsrSpot.mask.getMaskPlaneDict())
bit = 2**postIsrSpot.mask.getMaskPlaneDict()['CR']
# Or even better
bit = postIsrSpot.getMask().getPlaneBitMask('CR')

# Select interpolated pixels
sel = (postIsrSpot.mask.array & bit) > 0
sel.sum()

# Find a region to make cutouts around
idx,idy = np.where(sel)
print(idx)
print(idy)
print(postIsrSpot.mask.array[idx,idy])

# Make a cutout
fig,ax = plt.subplots(1,2,figsize=(8,4))
ax[0].imshow(postIsrSpot.image.array[1710:1720,645:665])
ax[1].imshow(postIsrSpot.mask.array[1710:1720,645:665])
